In [1]:
from preprocess import dataset,data_deal
from sklearn.preprocessing import MinMaxScaler
from models import *
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import KFold
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import json
import random
from TrainandTest import train_MLP,test_MLP
import pandas as pd


def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# Config

In [2]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
file = './data/total 12 data.xlsx'
data = pd.read_excel(file,engine="openpyxl")
data = np.array(data)
l = len(data)
para_path = './save/parameter'
batch_size = 300
plot = True
# np.set_printoptions(threshold=np.sys.maxsize)

# Train

In [3]:
def complexity(model):
    param_squared_sums = {}
    encoder_parameters = list(filter(lambda x: x[0].startswith('encoder'),list(model.named_parameters())))
    for name, param in encoder_parameters:
#         print(param)
        if 'weight' in name:
            param_squared_sums[name] = torch.sum(param ** 2)
    return sum(param_squared_sums.values())

In [4]:
total_loss = 0
flag =  0
Type = 'even'
N_size = '<=126'
lr = 0.1
k = 10
alpha = 0.1
input_size = 9
epsilon = 0
seed1,seed2 = 3063344997,173232029

In [5]:
kf = KFold(n_splits=k ,shuffle=True,random_state=seed1)

total_data = dataset(data,Type,N_size,input_size)
total_data = np.array(total_data,dtype = object)

X = total_data[:,0]
y = total_data[:,1].astype(np.float32)

temp_loss = 0
l1= X[0].shape[0]
l2 =len(X)
results_df = pd.DataFrame()

In [6]:
for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    ae_model = get_models("AE_MLP",input_size,epsilon)
    mlp_model = get_models("MLP",input_size,epsilon)
    print(f"ae init weight {complexity(ae_model)}")
    print(f"mlp init weight {complexity(mlp_model)}")
    scaler = MinMaxScaler()
    X_train,X_test,y_train,y_test,scaler = data_deal(X_train,X_test,y_train,y_test,scaler,input_size,flag)
    train_MLP(X_train,y_train,ae_model,lr,alpha,fold,para_path,flag)
    loss,_ = test_MLP(X_test,y_test,ae_model,fold,para_path,scaler,results_df,l1,flag)
    print(f"ae test loss:{loss.item()}")
    train_MLP(X_train,y_train,mlp_model,lr,alpha,fold,para_path,flag)
    loss,_ = test_MLP(X_test,y_test,mlp_model,fold,para_path,scaler,results_df,l1,flag)
    print(f"mlp test loss:{loss.item()}")
    print(f"ae trained weight {complexity(ae_model)}")
    print(f"mlp trained weight {complexity(mlp_model)}\n")

ae init weight 12.281000137329102
mlp init weight 9.813122749328613
ae test loss:0.659867525100708
mlp test loss:0.5233104825019836
ae trained weight -17.4729061126709
mlp trained weight 30.679094314575195

ae init weight 6.0461225509643555
mlp init weight 11.666089057922363
ae test loss:0.5129931569099426
mlp test loss:0.5225540995597839
ae trained weight 3.7200927734375
mlp trained weight 34.38715362548828

ae init weight 2.364917516708374
mlp init weight 7.701430320739746
ae test loss:0.10078362375497818
mlp test loss:0.07697702944278717
ae trained weight -4.682962417602539
mlp trained weight -19.521282196044922

ae init weight 5.264561653137207
mlp init weight 2.989790678024292
ae test loss:0.8276804089546204
mlp test loss:0.5544532537460327
ae trained weight -6.412235260009766
mlp trained weight -5.673384666442871

ae init weight 6.814538955688477
mlp init weight 7.049803733825684
ae test loss:0.4760608673095703
mlp test loss:0.7388693690299988
ae trained weight -7.89101505279541
